# BIG DATA [223090-0421]
## Projekt zaliczeniowy
Uladzislau Darhevich ud108519@student.sgh.waw.pl

# 1. Źródło danych

***Paycheck Protection Program (PPP) Loan Data***

Paycheck Protection Program (PPP) to program pożyczek biznesowych o wartości 953 miliardów dolarów, ustanowiony przez rząd federalny Stanów Zjednoczonych w 2020 roku na mocy ustawy *Coronavirus Aid, Relief, and Economic Security Act (CARES Act)*, aby pomóc niektórym przedsiębiorstwom, osobom samozatrudnionym, przedsiębiorcom jednoosobowym, niektórym organizacjom non-profit i przedsiębiorstwom plemiennym w dalszym płaceniu swoim pracownikom.

Oryginalny zbiór danych można znaleźć na [stronie](https://home.treasury.gov/policy-issues/cares-act/assistance-for-small-businesses/sba-paycheck-protection-program-loan-level-data) US Department of Treasury. W pracy zaliczeniowej użyto zagregowany plik .csv, który znajduje się [tu](https://www.kaggle.com/susuwatari/ppp-loan-data-paycheck-protection-program).

Udostępniając publicznie dane o pożyczkach PPP, [SBA](https://www.sba.gov/) - *small business administration* - zachowuje równowagę pomiędzy zapewnieniem przejrzystości amerykańskim podatnikom a ochroną poufnych informacji biznesowych małych firm, takich jak listy płac i dane osobowe. Małe firmy są siłą napędową amerykańskiej stabilności gospodarczej i mają zasadnicze znaczenie dla gospodarczego odbicia Ameryki po pandemii. SBA jest zobowiązane do zapewnienia, że jakiekolwiek ujawnienie danych o pożyczkach PPP nie zaszkodzi małym firmom ani ich pracownikom.

Pożyczki PPP nie są udzielane przez SBA. Kredyty PPP są udzielane przez instytucje kredytujące, a następnie gwarantowane przez SBA. W związku z tym, kredytobiorcy zgłaszają się do kredytodawców i sami poświadczają, że kwalifikują się do kredytów PPP. Autocertyfikacja obejmuje m.in. poświadczenie w dobrej wierze, że pożyczkobiorca ma potrzebę ekonomiczną wymagającą pożyczki oraz poświadczenie, że pożyczkobiorca zastosował zasady przynależności i jest małym przedsiębiorstwem. Pożyczkodawca następnie sprawdza wniosek pożyczkobiorcy i jeśli wszystkie formalności są w porządku, zatwierdza pożyczkę i przekazuje ją do SBA.

Mały biznes lub organizacja non-profit, która jest wymieniona w publicznie udostępnionych danych, została zatwierdzona do pożyczki PPP przez delegowanego pożyczkodawcę. Jednakże, zatwierdzenie pożyczkodawcy nie odzwierciedla ustalenia przez SBA, że pożyczkobiorca kwalifikuje się do pożyczki PPP lub jest uprawniony do umorzenia pożyczki. Wszystkie pożyczki PPP podlegają weryfikacji przez SBA, a wszystkie pożyczki powyżej $2 milionów będą automatycznie weryfikowane. Fakt, że pożyczkobiorca jest wymieniony w danych jako posiadający pożyczkę PPP nie oznacza, że SBA ustaliło, że pożyczkobiorca spełnia zasady programu lub jest uprawniony do otrzymania pożyczki PPP i umorzenia pożyczki. Ponadto, otrzymanie przez mały biznes pożyczki PPP nie powinno być interpretowane jako zatwierdzenie działalności biznesowej lub modelu biznesowego małego biznesu.

Publiczne dane PPP obejmują wyłącznie aktywne kredyty. Kredyty, które zostały anulowane z jakiegokolwiek powodu, nie są uwzględniane w publicznych danych.

Dane pożyczek PPP odzwierciedlają informacje, które pożyczkobiorcy dostarczyli swoim pożyczkodawcom ubiegając się o pożyczki PPP. SBA nie może składać żadnych oświadczeń co do dokładności lub kompletności informacji, które pożyczkobiorcy dostarczyli swoim pożyczkodawcom. Nie wszyscy pożyczkobiorcy przekazali wszystkie informacje. Na przykład, około 75% wszystkich pożyczek PPP nie zawierało żadnych informacji demograficznych, ponieważ informacje te nie zostały dostarczone przez pożyczkobiorców. SBA pracuje nad zebraniem większej ilości informacji demograficznych od pożyczkobiorców, aby lepiej zrozumieć, które małe firmy korzystają z pożyczek PPP. Aplikacja o umorzenie pożyczki wyraźnie wymaga od pożyczkobiorców informacji demograficznych.

In [0]:
#import bibliotek
from pyspark.sql.functions import isnan, when, count, col, mean, bround
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from urllib.request import urlopen

In [0]:
# importowanie danych
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/ud108519@student.sgh.waw.pl/PPP_data_150k_plus.csv", header="true")
display(df)

LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723,813920,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,295,04/14/2020,"National Cooperative Bank, National Association",AK - 00
a $5-10 million,CRUZ CONSTRUCTION INC,7000 East Palmer Wasilla Hwy,PALMER,AK,99645,238190,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,null,215,04/15/2020,First National Bank Alaska,AK - 00
a $5-10 million,"I. C. E. SERVICES, INC",2606 C Street,ANCHORAGE,AK,99503,722310,Corporation,Unanswered,Unanswered,Unanswered,null,367,04/11/2020,KeyBank National Association,AK - 00
a $5-10 million,KATMAI HEALTH SERVICES LLC,"11001 O'MALLEY CENTRE DRIVE, SUITE 204",ANCHORAGE,AK,99515,621111,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,null,0,04/29/2020,Truist Bank d/b/a Branch Banking & Trust Co,AK - 00
a $5-10 million,MATANUSKA TELEPHONE ASSOCIATION,1740 S. CHUGACH ST,PALMER,AK,99645,517311,Cooperative,Unanswered,Unanswered,Unanswered,null,267,06/10/2020,CoBank ACB,AK - 00
a $5-10 million,"NANA WORLEY, LLC","3700 Centerpoint Drive, 7th floor",ANCHORAGE,AK,99503,541330,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,null,231,05/19/2020,First National Bank Alaska,AK - 00
a $5-10 million,"NORTHERN ENERGY SERVICES, LLC",6250 South Airpark Place,ANCHORAGE,AK,99502,213112,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,null,298,04/06/2020,Northrim Bank,AK - 00
a $5-10 million,SOUTH PENINSULA HOSPITAL INC,4300 BARTLETT STREET,HOMER,AK,99603,622110,Corporation,Unanswered,Unanswered,Unanswered,null,439,04/14/2020,First National Bank Alaska,AK - 00
a $5-10 million,"TATITLEK TECHNOLOGIES, INC.",561 E. 36th Avenue,ANCHORAGE,AK,99503,518210,Corporation,Unanswered,Unanswered,Unanswered,null,361,04/14/2020,KeyBank National Association,AK - 00
a $5-10 million,"TYONEK GLOBAL SERVICES, LLC",1689 C ST,ANCHORAGE,AK,99501,611420,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,null,0,04/30/2020,BBVA USA,AK - 00


**Opis zmiennych:**
- LoanRange - Zakres zatwierdzonej pożyczki;
- BusinessName - Nazwa firmy;
- Address - Adres firmy;
- City - Miasto, w którym znajduje się firma;
- State - Stan, w którym znajduje się firma;
- Zip - Zip Code;
- NAICSCode - [North American Industry Classification System](https://pl.wikipedia.org/wiki/NAICS), statystyczna klasyfikacja działalności gospodarczych w Ameryce Północnej;
- BusinessType - Rodzaj działalności;
- RaceEthnicity - Rasa;
- Gender - Płeć;
- Veteran - Weteran;
- NonProfit - Czy jest organizacją non-profit;
- JobsRetained - Ile miejsc pracy pozostało po zatwierdzeniu kredytu;
- DateApproved - Data zatwierdzenia;
- Lender - Bank Kredytowy;
- CD - Business Congressional District. 


# 2. Podstawowe transformacje danych

In [0]:
# Rozmiar zbióra danych
print("Zbiór danych zawiera {} wierszy oraz {} kolumn.". format(df.count(), len(df.columns)))

Zbiór danych zawiera 661218 wierszy oraz 16 kolumn.

In [0]:
#typy danych dla każdej kolumny
df.printSchema()

root
-- LoanRange: string (nullable = true)
-- BusinessName: string (nullable = true)
-- Address: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- Zip: string (nullable = true)
-- NAICSCode: string (nullable = true)
-- BusinessType: string (nullable = true)
-- RaceEthnicity: string (nullable = true)
-- Gender: string (nullable = true)
-- Veteran: string (nullable = true)
-- NonProfit: string (nullable = true)
-- JobsRetained: string (nullable = true)
-- DateApproved: string (nullable = true)
-- Lender: string (nullable = true)
-- CD: string (nullable = true)

In [0]:
# Ilość braków danych dla każdej kolumny
count_na = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
count_na.show()

+---------+------------+-------+----+-----+---+---------+------------+-------------+------+-------+---------+------------+------------+------+---+
LoanRange|BusinessName|Address|City|State|Zip|NAICSCode|BusinessType|RaceEthnicity|Gender|Veteran|NonProfit|JobsRetained|DateApproved|Lender| CD|
+---------+------------+-------+----+-----+---+---------+------------+-------------+------+-------+---------+------------+------------+------+---+
 0| 0| 0| 0| 0| 16| 6782| 1430| 0| 0| 0| 618754| 40508| 0| 0| 0|
+---------+------------+-------+----+-----+---+---------+------------+-------------+------+-------+---------+------------+------------+------+---+

In [0]:
#Kolumna `NonProfit` zawiera wartośći: "null", "Y", "Unanswered". Wartość "null" zostanie zamieniona na "N" 
df.groupby("NonProfit").count().show()

+----------+------+
 NonProfit| count|
+----------+------+
 null|618754|
 Y| 42462|
Unanswered| 2|
+----------+------+

In [0]:
# Zmiana typów danych dla kolumn `JobsRetained` oraz `DateApproved`
df = df.withColumn("JobsRetained", col("JobsRetained").cast("Integer"))
df = df.withColumn("DateApproved", to_date("DateApproved", 'MM/dd/yyyy'))

In [0]:
#Uzupełnienie braków danych
JobsRetained_mean = df.agg(bround(mean("JobsRetained"), 3)).first()[0]

df = df.na.fill({'Zip': 'unknown', 
                 'NAICSCode': 'unknown',
                 'BusinessType' : 'unknown',
                 'NonProfit' : 'N',
                 'JobsRetained' : JobsRetained_mean})

In [0]:
#usunięcie liter na początku `LoanRange`
df = df.withColumn('LoanRange', regexp_replace('LoanRange', '^(\w *?)\s*', ''))

In [0]:
# Zmienna `CD` nie daje ważnej informacji i dublikuje kolumny `State` oraz 'City' i będzie usunięta
df = df.drop('CD')

# 3. Analiza eksploracyjna danych (wykresy i tabele)

In [0]:
# kategorie pożyczek
display(df.groupBy("LoanRange").count())

LoanRange,count
$2-5 million,24838
$1-2 million,53030
"$150,000-350,000",379054
$5-10 million,4840
"$350,000-1 million",199456


In [0]:
#Mapa z ilością pożyczek dla każdego stanu
display(df.groupby('State').count())

State,count
AZ,11326
CA,87689
AL,7872
AS,19
AK,1652
AR,4256
DC,2806
CT,8595
DE,2073
CO,13385


In [0]:
#Top-20 miast z najwyższej ilością pozyczek
display(df.groupBy('City').count().orderBy('Count', ascending=False).take(20))

City,count
NEW YORK,12851
HOUSTON,8497
CHICAGO,6248
LOS ANGELES,5690
DALLAS,4611
MIAMI,4203
BROOKLYN,4062
SAN FRANCISCO,3868
SAN DIEGO,3724
ATLANTA,3686


Dla dalszej analizy dataframe `df` został zapisany  w bazie danych Hive, w tabeli `ppp`

In [0]:
df.createOrReplaceTempView('ppp')

Niżej przedstawiono dane o ilości kredytów, średniej oraz ogólnej ilości zatrzymanych miejsc pracy dla każdej kategorii. Też podane ile procent wszystkich zapisanych miejsc pracy zajmuje każda kategoria. Z tabeli widać, że mały i średni biznes (od 150k do 1M) generuje więcej niż 50% zatrzymanych miejsc. *Ogólnie w Stanach zatrudniono 159,932,000 osób (czerwiec 2020)*

In [0]:
%sql

select LoanRange, count(*) as Number_of_loans, 
                  round(avg(JobsRetained), 2) as Avg_JobsRetained, 
                  sum(JobsRetained) as Total_JobsRetained,
                  round(sum(JobsRetained) *100 / SUM(SUM(JobsRetained)) over (), 2) as Percentage
from ppp
group by LoanRange

LoanRange,Number_of_loans,Avg_JobsRetained,Total_JobsRetained,Percentage
$2-5 million,24838,211.26,5247394,15.67
$1-2 million,53030,114.53,6073594,18.14
"$150,000-350,000",379054,26.08,9885373,29.52
$5-10 million,4840,341.81,1654376,4.94
"$350,000-1 million",199456,53.25,10621130,31.72


Instytucje kredytowe, które udzieliły najwięcej kredytów. Pożyczki PPP nie są udzielane przez SBA. Kredyty PPP są udzielane przez banki, fundusze i td.

In [0]:
%sql

select Lender, count(*) as Num_of_loans
from ppp
group by Lender
order by Num_of_loans desc
limit 10


Lender,Num_of_loans
"JPMorgan Chase Bank, National Association",36698
"Bank of America, National Association",29157
Truist Bank d/b/a Branch Banking & Trust Co,16076
"PNC Bank, National Association",14649
"Wells Fargo Bank, National Association",12147
"TD Bank, National Association",10389
KeyBank National Association,9352
"U.S. Bank, National Association",8658
"Zions Bank, A Division of",8418
Manufacturers and Traders Trust Company,8345


Poniższy wykres dobrze ilustruje różne fazy PPP:

- Termin nadsyłania zgłoszeń: 04/03/2020 - 04/12/2020 (w ciągu 9 dni od rozpoczęcia programu)
- Okresy szczytowe: 04/13/2020 - 05/03/2020 (rozpoczął się, gdy SBA ogłosił nowe wytyczne dla wniosków)
- Okres "ease out" : 05/04/2020-06/11/2020
- Okres przebudowy: 06/12/2020-wstępny termin; 06/30/2020
- Spóźnione zgłoszenia: 06/22/2020-początkowy termin; 06/30/2020 (w ciągu 8 dni do początkowego terminu programu)

In [0]:
%sql

select DateApproved, count(*) as Num_of_approved
from ppp
group by DateApproved
order by DateApproved 

DateApproved,Num_of_approved
null,2
2020-04-03,9346
2020-04-04,16104
2020-04-05,19127
2020-04-06,27325
2020-04-07,36533
2020-04-08,34620
2020-04-09,37698
2020-04-10,38513
2020-04-11,29525


Dalej badano, które branże uzyskały najwięcej zatwierdzonych pożyczek, również jak wygląda sytuacja w każdej kategorii (top-5 dla każdej)

In [0]:
#import tabeli z kodami, źródło - https://www.census.gov/eos/www/naics/downloadables/downloadables.html
codes = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/ud108519@student.sgh.waw.pl/codes-1.csv", header="true")
codes = codes.withColumnRenamed("2017 NAICS Code","code").withColumnRenamed("2017 NAICS Title","title")
codes.createOrReplaceTempView('codes')

In [0]:
%sql

--top-10

select c.title, ppp.NAICSCode, count(*) as Num_of_loans
from ppp
join codes c on ppp.NAICSCode = c.code
group by c.title, ppp.NAICSCode
order by Num_of_loans desc
limit 10

title,NAICSCode,Num_of_loans
Full-Service Restaurants,722511,33608
Offices of Physicians (except Mental Health Specialists),621111,20932
Offices of Lawyers,541110,14310
"Plumbing, Heating, and Air-Conditioning Contractors",238220,13140
New Car Dealers,441110,12694
Commercial and Institutional Building Construction,236220,10942
Religious Organizations,813110,10681
Offices of Dentists,621210,10627
Limited-Service Restaurants,722513,10207
Electrical Contractors and Other Wiring Installation Contractors,238210,9886


In [0]:
%sql
select * 
from
    (select ppp.LoanRange, c.title, ppp.NAICSCode, count(*) as Num_of_loans, 
            row_number() over (partition by ppp.LoanRange order by count(*) desc) as my_rank 
    from ppp join codes c on ppp.NAICSCode = c.code
    group by ppp.LoanRange, c.title, ppp.NAICSCode) foo
    
where my_rank <= 5

LoanRange,title,NAICSCode,Num_of_loans,my_rank
$1-2 million,New Car Dealers,441110,2647,1
$1-2 million,Offices of Physicians (except Mental Health Specialists),621111,1476,2
$1-2 million,Nursing Care Facilities (Skilled Nursing Facilities),623110,1192,3
$1-2 million,Elementary and Secondary Schools,611110,1018,4
$1-2 million,Full-Service Restaurants,722511,1004,5
"$150,000-350,000",Full-Service Restaurants,722511,24631,1
"$150,000-350,000",Offices of Physicians (except Mental Health Specialists),621111,12307,2
"$150,000-350,000",Offices of Dentists,621210,8990,3
"$150,000-350,000",Offices of Lawyers,541110,8620,4
"$150,000-350,000","Plumbing, Heating, and Air-Conditioning Contractors",238220,7502,5


Jakie typy firm uzyskali największą ilość kredytów?

In [0]:
%sql

select BusinessType, count(*) as Num_of_loans, round(count(*) *100 / sum(count(*)) over(), 2) as Percentage
from ppp
group by BusinessType
order by Num_of_loans desc

BusinessType,Num_of_loans,Percentage
Corporation,275482,41.66
Limited Liability Company(LLC),172643,26.11
Subchapter S Corporation,132434,20.03
Non-Profit Organization,41819,6.32
Partnership,12902,1.95
Sole Proprietorship,8774,1.33
Limited Liability Partnership,7649,1.16
Professional Association,3858,0.58
Cooperative,1851,0.28
unknown,1430,0.22


# 4. Podsumowanie


Na podstawie danych i przedstawionej analizy zatwierdzonych pożyczek można zrozumieć, w jaki sposób rozdzielono pomoc finansową mającą na celu przezwyciężenie skutków Covid-19 w Stanach Zjednoczonych. Większość uratowanych miejsc pracy znajduje się w małych i średnich przedsiębiorstwach. Najwięcej kredytów udzieliły główne amerykańskie instytucje finansowe (JPMorgan Chase Bank oraz Bank of America). Większość firm złożyła wniosek o udział w programie PPP w terminie. Najwięcej kredytów otrzymała branża gastronomiczna, gabinety lekarskie, kancelarie adwokackie, firmy budowlane.